# Entrenamiento RN

In [76]:
import pandas as pd
import numpy as np

import missingno as msno
import matplotlib.pyplot as plt
import plotly.express as px

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split 
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
import datetime
from dateutil.parser import parse
from sklearn.tree import  DecisionTreeClassifier
import plotly.graph_objects as go

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
import xgboost as xgb
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
import ml_metrics
import pickle
from sklearn.metrics import accuracy_score
import joblib
import json
import sklearn.metrics as metrics
from keras.models import Sequential
from keras.layers import Dense
#from keras.optimizers import SGD, Adam
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from keras.callbacks import TensorBoard
from keras.callbacks import EarlyStopping
import warnings
from tensorflow import keras
from plotly.subplots import make_subplots

warnings.filterwarnings("ignore")

In [2]:
Data=pd.read_csv("cleaned_05June2022.csv").drop(columns=["Unnamed: 0"]).fillna("")

In [3]:
# datos númericos
numerical=list(Data.select_dtypes(exclude=["category","object"]).columns)
numerical.remove("Sexo")
numerical.remove('Recibe_sueldo_en_cuenta')


# datos categorical
categorical=list(Data.select_dtypes(include=["category","object"]).columns)+['Sexo','Recibe_sueldo_en_cuenta']
categorical.remove("Target")

In [4]:
for i in categorical:
    Data[i]=Data[i].astype(str)

In [5]:
# Select 1 - Only Gini, corte visual
select_variables_1=['E-F',
 'D-F',
 'C-C',
 'A-K',
 'A-G',
 'num_comunicaciones',
 'Recency',
 'Positivo',
 'B',
 'Monto',
 'Periodicity',
 'Profesion',
 'lec_B-B',
 'Renta',
 'F-I',
 'Edad',
 'lec_A-A',
 'Comuna',
 'Sin Info',
 'Ciudad',
 'Duracion_Campaña',
 'Principalidad']

In [6]:
numerical1=list(filter(lambda x:x  in numerical,select_variables_1))
categorical1=list(filter(lambda x:x  in categorical,select_variables_1))

In [7]:
# Preprocesamiento de datos
preprocessing_transformer1 = ColumnTransformer(
    transformers=[('OneHotEncoder', OneHotEncoder(handle_unknown="ignore"),categorical1 ),
        ('MinMax', MinMaxScaler(),numerical1),
    ])

In [8]:
pipe1 = Pipeline(
    [("preprocesamiento", preprocessing_transformer1)]
)

------------------------

# Train, Test Split

In [9]:
X=Data.drop(columns=["Target"])[select_variables_1].copy()

In [10]:
Y=Data["Target"]

In [11]:
enc = OneHotEncoder()
Y = enc.fit_transform(Y[:, np.newaxis]).toarray()

In [12]:
dic={}
to_enc_inv=list(enc.categories_[0])
for i in range(len(to_enc_inv)):
    dic[i]=to_enc_inv[i]

In [13]:
inv_map = {v: k for k, v in dic.items()}

In [14]:
# Split the data set into training and testing
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.33, random_state=2)

In [15]:
# Sobremuestreo:
ros=RandomOverSampler(random_state=0) # Random_state=0
# ROS:
X_train, Y_train=ros.fit_resample(X_train,Y_train)

In [16]:
X_test.shape

(21460, 22)

In [17]:
# Preprocesamiento
pipe1.fit(X_train)
X_train1=pipe1.transform(X_train)
X_test1=pipe1.transform(X_test)
n_features1 =X_train1.shape[1]
n_classes1 =Y_train.shape[1]

#### Definición de arquitectura

In [50]:
model1 = Sequential()
model1.add(Dense(4, input_dim=n_features1, activation='relu'))
model1.add(Dense(n_classes1,activation='softmax'))
model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 

#alfa=0.001
#my_optimizer = SGD(lr = alfa)
#model.compile(optimizer=my_optimizer,loss='categorical_crossentropy',metrics=['accuracy'])
model1.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_11 (Dense)            (None, 4)                 2344      
                                                                 
 dense_12 (Dense)            (None, 35)                175       
                                                                 
Total params: 2,519
Trainable params: 2,519
Non-trainable params: 0
_________________________________________________________________


In [51]:
cb = TensorBoard(log_dir='./', histogram_freq=1)
early_stopping_monitor = EarlyStopping(monitor='val_loss', patience=10)

In [52]:
history_callback=model1.fit(X_train1[0:30000], Y_train[0:30000], epochs=1000, batch_size=10,verbose=1,
validation_data=(X_test1, Y_test),
callbacks=[early_stopping_monitor],
#validation_split = 0.1
#callbacks=[cb]
)

Epoch 1/1000
3000/3000 [==============================] - 16s 5ms/step - loss: 2.8442 - accuracy: 0.2202 - val_loss: 2.4744 - val_accuracy: 0.2928
Epoch 2/1000
3000/3000 [==============================] - 13s 4ms/step - loss: 2.3341 - accuracy: 0.3110 - val_loss: 2.2162 - val_accuracy: 0.3351
Epoch 3/1000
3000/3000 [==============================] - 11s 4ms/step - loss: 2.1522 - accuracy: 0.3448 - val_loss: 2.0904 - val_accuracy: 0.3589
Epoch 4/1000
3000/3000 [==============================] - 11s 4ms/step - loss: 2.0553 - accuracy: 0.3638 - val_loss: 2.0171 - val_accuracy: 0.3780
Epoch 5/1000
3000/3000 [==============================] - 11s 4ms/step - loss: 1.9947 - accuracy: 0.3750 - val_loss: 1.9723 - val_accuracy: 0.3879
Epoch 6/1000
3000/3000 [==============================] - 14s 5ms/step - loss: 1.9544 - accuracy: 0.3854 - val_loss: 1.9395 - val_accuracy: 0.3975
Epoch 7/1000
3000/3000 [==============================] - 16s 5ms/step - loss: 1.9251 - accuracy: 0.3948 - val_loss: 1

In [53]:
scores = model1.evaluate(X_test1, Y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])
print(scores)

671/671 [==============================] - 2s 3ms/step - loss: 1.7893 - accuracy: 0.4414
Test loss: 1.7892714738845825
Test accuracy: 0.4413793087005615
[1.7892714738845825, 0.4413793087005615]


In [54]:
  ### Genero la predicción con el modelo
y_pred = model1.predict(X_test1)

671/671 [==============================] - 2s 2ms/step


In [55]:
y_pred_1col = np.argmax(y_pred, axis=1, out=None) 

In [56]:
y_pred_1col = pd.DataFrame(y_pred_1col, columns= ['target'])

In [57]:
y_test_1col = pd.DataFrame(enc.inverse_transform(Y_test), columns= ['target']).replace(inv_map)

In [58]:
Ypred_map=y_pred_1col["target"].replace(dic).apply( lambda x: x.split(" "))
Ytest_map=y_test_1col["target"].replace(dic).apply( lambda x: x.split(" "))
ml_metrics.mapk(Ytest_map, Ypred_map, 5)

0.6912744641192917

In [75]:
range(len(history_dict['accuracy']))

range(0, 86)

In [87]:
# Visualización
history_dict=history_callback.history
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=("Accuracy", "Cross-Entropy"))
# Create traces

fig.add_trace(go.Scatter(x=list(range(len(history_dict['accuracy']))), y=history_dict['val_accuracy'],
                    mode='lines',
                    name='Val_accuracy'),row=1, col=1)
fig.add_trace(go.Scatter(x=list(range(len(history_dict['accuracy']))), y=history_dict['accuracy'],
                    mode='lines',
                    name='Accuracy'),row=1, col=1)

fig.add_trace(go.Scatter(x=list(range(len(history_dict['accuracy']))), y=history_dict['val_loss'],
                    mode='lines',
                    name='Val_loss'),row=2, col=1)

fig.add_trace(go.Scatter(x=list(range(len(history_dict['accuracy']))), y=history_dict['loss'],
                    mode='lines',
                    name='Loss'),row=2, col=1)

fig.update_layout(title_text="Resultados entrenamiento de la Red Neuronal",
                      height=800,
                      width=1000,font=dict(#color="white"
                      ),#template="plotly_dark",
                      legend_tracegroupgap = 90,legend_title="Leyenda")

fig.update_xaxes(title_text="Epoch",row=2, col=1)
fig.update_yaxes(title_text="Accuracy", row=1, col=1)
fig.update_yaxes(title_text="Loss", row=2, col=1)

fig.show()

--------------------

In [60]:
# Guardar pipe:
pickle.dump(pipe1, open('pipe_rn.pkl', "wb"))

In [64]:
# Leer pipe:
pipe1= pickle.load(open('pipe_rn.pkl', 'rb'))

In [61]:
# Guardar enc:
pickle.dump(enc, open('encoder.pkl', "wb"))

In [65]:
# Leer enc:
enc= pickle.load(open('encoder.pkl', 'rb'))

In [62]:
# Guardar el Modelo
model1.save('my_model1.h5')



In [66]:
# Recrea exactamente el mismo modelo solo desde el archivo
model1 = keras.models.load_model('my_model1.h5')

model1.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_11 (Dense)            (None, 4)                 2344      
                                                                 
 dense_12 (Dense)            (None, 35)                175       
                                                                 
Total params: 2,519
Trainable params: 2,519
Non-trainable params: 0
_________________________________________________________________


----------------------

In [67]:
  ### Genero la predicción con el modelo
y_pred = model1.predict(X_test1)

671/671 [==============================] - 2s 2ms/step


In [68]:
y_pred_1col = np.argmax(y_pred, axis=1, out=None) 

In [69]:
y_pred_1col = pd.DataFrame(y_pred_1col, columns= ['target'])

In [70]:
y_test_1col = pd.DataFrame(enc.inverse_transform(Y_test), columns= ['target']).replace(inv_map)

In [71]:
Y_test

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [72]:
Ypred_map=y_pred_1col["target"].replace(dic).apply( lambda x: x.split(" "))
Ytest_map=y_test_1col["target"].replace(dic).apply( lambda x: x.split(" "))
ml_metrics.mapk(Ytest_map, Ypred_map, 5)

0.6912744641192917

In [73]:
scores = model1.evaluate(X_test1, Y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])
print(scores)

671/671 [==============================] - 2s 2ms/step - loss: 1.7893 - accuracy: 0.4414
Test loss: 1.7892714738845825
Test accuracy: 0.4413793087005615
[1.7892714738845825, 0.4413793087005615]
